In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import csv
import requests
from config import api_key_omdb2
import time
import gzip
import shutil

# First API call and cleanup

In [2]:
#Load in csv with IMBD ids found on Kaggle
#(https://www.kaggle.com/rounakbanik/the-movies-dataset#links.csv)
omdb_link_file = pd.read_csv('.\\raw_data\\links.csv')
omdb_link_data = pd.DataFrame(omdb_link_file)

#Add leading zeros and tt to the IMBD id column
omdb_link_data['imdbId'] = omdb_link_data['imdbId'].apply(lambda x: 'tt{0:0>7}'.format(x))

#Convert omdb column to list
omdb_id_list = omdb_link_data['imdbId'].tolist()

len(omdb_id_list)

45843

In [3]:
#Set up for call
url = "http://private.omdbapi.com/"
params = {"apikey":api_key_omdb2}
not_found = []

#Set up loop
counter = 0
start_time = time.time()
for x in range (len(omdb_id_list)):
    
    #movie paramater is selected from list
    movieid = omdb_id_list[x]
    params["i"] = movieid
    
    response = requests.get(url,params=params).json()
    try:
        # assemble url and make API request
        omdb_link_data.loc[x,'Title'] = response.get("Title",'')
        omdb_link_data.loc[x,'Year'] = response.get("Year",'')
        omdb_link_data.loc[x,'Released'] = response.get("Released",'')
        omdb_link_data.loc[x,'Runtime'] = response.get("Runtime",'')
        omdb_link_data.loc[x,'Genre'] = response.get("Genre",'')
        omdb_link_data.loc[x,'Director'] = response.get("Director",'')
        omdb_link_data.loc[x,'Writer'] = response.get("Writer",'')
        omdb_link_data.loc[x,'Actors'] = response.get("Actors",'')
        omdb_link_data.loc[x,'Plot'] = response.get("Plot",'')
        omdb_link_data.loc[x,'Country'] = response.get("Country",'')
        omdb_link_data.loc[x,'Awards'] = response.get("Awards",'')
        omdb_link_data.loc[x,'RT_Ratings'] = response["Ratings"][1].get("Value",'')
        omdb_link_data.loc[x,'Metascore'] = response.get("Metascore",'')
        omdb_link_data.loc[x,'imdbRating'] = response.get("imdbRating",'')
        omdb_link_data.loc[x,'imdbVotes'] = response.get("imdbVotes",'')
        omdb_link_data.loc[x,'imdbID'] = response.get("imdbID",'')
        omdb_link_data.loc[x,'Type'] = response.get("Type",'')
        omdb_link_data.loc[x,'BoxOffice'] = response.get("BoxOffice",'')
        omdb_link_data.loc[x,'Production'] = response.get("Production",'')
        
    except (KeyError, ValueError) as e:
        print(f"skipping {movieid}")
        not_found.append(movieid)
        
    #If there is no RT_rating
    except IndexError:
        omdb_link_data.loc[x,'Title'] = response.get("Title",'')
        omdb_link_data.loc[x,'Year'] = response.get("Year",'')
        omdb_link_data.loc[x,'Released'] = response.get("Released",'')
        omdb_link_data.loc[x,'Runtime'] = response.get("Runtime",'')
        omdb_link_data.loc[x,'Genre'] = response.get("Genre",'')
        omdb_link_data.loc[x,'Director'] = response.get("Director",'')
        omdb_link_data.loc[x,'Writer'] = response.get("Writer",'')
        omdb_link_data.loc[x,'Actors'] = response.get("Actors",'')
        omdb_link_data.loc[x,'Plot'] = response.get("Plot",'')
        omdb_link_data.loc[x,'Country'] = response.get("Country",'')
        omdb_link_data.loc[x,'Awards'] = response.get("Awards",'')
        omdb_link_data.loc[x,'Metascore'] = response.get("Metascore",'')
        omdb_link_data.loc[x,'imdbRating'] = response.get("imdbRating",'')
        omdb_link_data.loc[x,'imdbVotes'] = response.get("imdbVotes",'')
        omdb_link_data.loc[x,'imdbID'] = response.get("imdbID",'')
        omdb_link_data.loc[x,'Type'] = response.get("Type",'')
        omdb_link_data.loc[x,'BoxOffice'] = response.get("BoxOffice",'')
        omdb_link_data.loc[x,'Production'] = response.get("Production",'')
        
    #Counter added so API call is not redone        
    counter+=1        
    if counter == 10:
        break
end_time = time.time()
elapsed = end_time - start_time
print(f"{elapsed} seconds elapsed, {len(not_found)} record(s) not found")

#omdb_link_data.to_csv('.\\raw_data\\raw_omdb.csv', encoding="utf-8", index=False)

1.4145877361297607 seconds elapsed, 0 record(s) not found


In [4]:
omdb_file = pd.read_csv('.\\raw_data\\raw_omdb.csv')
omdb_data = pd.DataFrame(omdb_file)

In [5]:
omdb_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45843 entries, 0 to 45842
Data columns (total 22 columns):
movieId       45843 non-null int64
imdbId        45843 non-null object
tmdbId        45624 non-null float64
Title         45842 non-null object
Year          45842 non-null object
Released      44060 non-null object
Runtime       45338 non-null object
Genre         45650 non-null object
Director      45254 non-null object
Writer        42894 non-null object
Actors        44998 non-null object
Plot          44213 non-null object
Country       45728 non-null object
Awards        25722 non-null object
RT_Ratings    19457 non-null object
Metascore     11586 non-null float64
imdbRating    45712 non-null float64
imdbVotes     45711 non-null object
imdbID        45842 non-null object
Type          45842 non-null object
BoxOffice     5433 non-null object
Production    30586 non-null object
dtypes: float64(3), int64(1), object(18)
memory usage: 7.7+ MB


In [6]:
#Function to correct problems with RT_ratings series
#Split as string on unwanted character, return value before character
def ratings_parse(x):
    if '%' in x:
        return x.split('%')[0]
    elif '/' in x:
        return x.split('/')[0]
    elif '–' in x:
        return x.split('–')[0]
    elif x is None:
        return None

In [7]:
#Filter out any series or episode
omdb_movie = omdb_data[omdb_data["Type"]=='movie']

In [8]:
#Filter out titles that contain #DUPE#
omdb_movies = omdb_movie[omdb_movie.Title != '#DUPE#']

In [9]:
#Retain only needed column for movie analysis
mod_omdb_movie_df = omdb_movies[['Title','Year','Genre','Country','RT_Ratings','Metascore','imdbRating','imdbVotes','Type','Production']]

In [10]:
#Filter out movies from before 1970
mod_omdb_movie_df= mod_omdb_movie_df[mod_omdb_movie_df["Year"] >= '1970']

In [11]:
#Prepare for applying function
mod_omdb_movie_rt_ratings= mod_omdb_movie_df[mod_omdb_movie_df["RT_Ratings"].notnull()]

In [12]:
#Normalize Rotten Tomatoes ratings
mod_omdb_movie_rt_ratings["RT_Ratings2"] = mod_omdb_movie_rt_ratings["RT_Ratings"].apply(ratings_parse)

C:\Users\bethf\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [13]:
#Normalize imdb Ratings
mod_omdb_movie_rt_ratings["imdbRating2"] = mod_omdb_movie_rt_ratings["imdbRating"].multiply(10)

C:\Users\bethf\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [14]:
#Save as movies CSV
mod_omdb_movie_rt_ratings.to_csv('.\\cleaned_data\\omdb_movies_cleaned.csv', encoding="utf-8", index=False)

# Second API call and cleanup

In [15]:
#https://www.imdb.com/interfaces/
#Unzip files and save as CSV
with gzip.open('.\\raw_data\\title.akas.tsv.gz', 'rb') as f_in:
    with open('.\\raw_data\\title.csv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
        
with gzip.open('.\\raw_data\\title.basics.tsv.gz', 'rb') as f_in:
    with open('.\\raw_data\\titlebasics.csv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
        
with gzip.open('.\\raw_data\\title.ratings.tsv.gz', 'rb') as f_in:
    with open('.\\raw_data\\titleratings.csv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [16]:
#Make list of ids for tv titles with release year since 1970 and movies released in 2018
tvtitles = []
movietitles2018 = []
years = ['1970','1971','1972','1973','1974','1975','1976','1977','1978','1979',\
         '1980','1981','1982','1983','1984','1985','1986','1987','1988','1989',\
         '1990','1991','1992','1993','1994','1995','1996','1997','1998','1999',\
         '2000','2001','2002','2003','2004','2005','2006','2007','2008','2009',\
         '2010','2011','2012','2013','2014','2015','2016','2017','2018']

with open('.\\raw_data\\titlebasics.csv',newline='',encoding='utf8',errors='ignore') as f:
    data = csv.reader(f, delimiter='\t')
    csv_header = next(data)
    
    #This retrieves IDs for all tv Series and movies from 2018
    for row in data:
        if row[1] == 'tvSeries' and row[5] in years:
            tvtitles.append(row[0])
        if row[1] == 'movie' and row[5] == '2018':
            movietitles2018.append(row[0])

In [17]:
len(tvtitles)

142166

In [18]:
type(movietitles2018)

list

In [19]:
#Retrieve number of votes to concatenate with above retrieved and narrow down lists before API call
#Warning to Alex: this cell takes a long time to run
dfm = []
dft = []

with open('.\\raw_data\\titleratings.csv',newline='',encoding='utf8',errors='ignore') as f:
    data = csv.reader(f, delimiter='\t')
    csv_header = next(data)

    for row in data:
        if row[0] in movietitles2018:
            dfm.append((row[0], row[2]))
            
        if row[0] in tvtitles:
            dft.append((row[0], row[2]))

In [20]:
#Make dataframes from retrieved data
movie_df = DataFrame(dfm)
movie_df.columns = ['ID','Votes']

tv_df = DataFrame(dft)
tv_df.columns = ['ID','Votes']

movie_df.to_csv('.\\raw_data\\movie_2018id.csv', encoding="utf-8", index=False)
tv_df.to_csv('.\\raw_data\\tvseriesid.csv', encoding="utf-8", index=False)

In [21]:
movie_file = pd.read_csv('.\\raw_data\\movie_2018id.csv')
movie_df = DataFrame(movie_file)

#Filter out movies with less than 500 votes
movie_df['Votes'] = movie_df['Votes'].astype('int64')
movie_df = movie_df[movie_df.Votes > 500]
movie_df.reset_index(inplace=True,drop=True)

#Convert movie id column to list
movie_id_list = movie_df['ID'].tolist()
len(movie_id_list)

1128

In [22]:
#Movies from 2018 and more than 500 votes API call
url = "http://private.omdbapi.com/"
params = {"apikey":api_key_omdb2}
not_found = []

counter = 0
start_time = time.time()
for x in range (len(movie_id_list)):

    movieid = movie_id_list[x]
    params["i"] = movieid
    
    response = requests.get(url,params=params).json()
    try:
        # assemble url and make API request
        movie_df.loc[x,'Title'] = response.get("Title",'')
        movie_df.loc[x,'Year'] = response.get("Year",'')
        movie_df.loc[x,'Released'] = response.get("Released",'')
        movie_df.loc[x,'Runtime'] = response.get("Runtime",'')
        movie_df.loc[x,'Genre'] = response.get("Genre",'')
        movie_df.loc[x,'Director'] = response.get("Director",'')
        movie_df.loc[x,'Writer'] = response.get("Writer",'')
        movie_df.loc[x,'Actors'] = response.get("Actors",'')
        movie_df.loc[x,'Plot'] = response.get("Plot",'')
        movie_df.loc[x,'Country'] = response.get("Country",'')
        movie_df.loc[x,'Awards'] = response.get("Awards",'')
        movie_df.loc[x,'RT_Ratings'] = response["Ratings"][1].get("Value",'')
        movie_df.loc[x,'Metascore'] = response.get("Metascore",'')
        movie_df.loc[x,'imdbRating'] = response.get("imdbRating",'')
        movie_df.loc[x,'imdbVotes'] = response.get("imdbVotes",'')
        movie_df.loc[x,'imdbID'] = response.get("imdbID",'')
        movie_df.loc[x,'Type'] = response.get("Type",'')
        movie_df.loc[x,'BoxOffice'] = response.get("BoxOffice",'')
        movie_df.loc[x,'Production'] = response.get("Production",'')
        
    except (KeyError, ValueError) as e:
        print(f"skipping {movieid}")
        not_found.append(movieid)
    except IndexError:
        movie_df.loc[x,'Title'] = response.get("Title",'')
        movie_df.loc[x,'Year'] = response.get("Year",'')
        movie_df.loc[x,'Released'] = response.get("Released",'')
        movie_df.loc[x,'Runtime'] = response.get("Runtime",'')
        movie_df.loc[x,'Genre'] = response.get("Genre",'')
        movie_df.loc[x,'Director'] = response.get("Director",'')
        movie_df.loc[x,'Writer'] = response.get("Writer",'')
        movie_df.loc[x,'Actors'] = response.get("Actors",'')
        movie_df.loc[x,'Plot'] = response.get("Plot",'')
        movie_df.loc[x,'Country'] = response.get("Country",'')
        movie_df.loc[x,'Awards'] = response.get("Awards",'')
        movie_df.loc[x,'Metascore'] = response.get("Metascore",'')
        movie_df.loc[x,'imdbRating'] = response.get("imdbRating",'')
        movie_df.loc[x,'imdbVotes'] = response.get("imdbVotes",'')
        movie_df.loc[x,'imdbID'] = response.get("imdbID",'')
        movie_df.loc[x,'Type'] = response.get("Type",'')
        movie_df.loc[x,'BoxOffice'] = response.get("BoxOffice",'')
        movie_df.loc[x,'Production'] = response.get("Production",'')
        
    counter+=1        
    if counter == 10:
        break            
end_time = time.time()
elapsed = end_time - start_time
print(f"{elapsed} seconds elapsed, {len(not_found)} record(s) not found")
#movie_df.to_csv('.\\raw_data\\movies2018_raw.csv', encoding="utf-8", index=False)

0.9896650314331055 seconds elapsed, 0 record(s) not found


# Third API and cleanup

In [23]:
tv_df = pd.read_csv('.\\raw_data\\tvseriesid.csv')

#Filter out series with less than 100 votes
tv_df['Votes'] = tv_df['Votes'].astype('int64')
tv_df = tv_df[tv_df.Votes > 100]
tv_df.reset_index(inplace=True,drop=True)

#Convert movie id column to list
tv_id_list = tv_df['ID'].tolist()
len(tv_id_list)

14942

In [24]:
#TV Series 1970-2018 with greater than 100 votes API call
#RT_Ratings are not found for TV Series, so index error handling is not necessary

not_found = []

start_time = time.time()
for x in range (len(tv_id_list)):

    tvid = tv_id_list[x]
    params["i"] = tvid
    
    response = requests.get(url,params=params).json()
    try:
        # assemble url and make API request
        tv_df.loc[x,'Title'] = response.get("Title",'')
        tv_df.loc[x,'Year'] = response.get("Year",'')
        tv_df.loc[x,'Released'] = response.get("Released",'')
        tv_df.loc[x,'Runtime'] = response.get("Runtime",'')
        tv_df.loc[x,'Genre'] = response.get("Genre",'')
        tv_df.loc[x,'Director'] = response.get("Director",'')
        tv_df.loc[x,'Writer'] = response.get("Writer",'')
        tv_df.loc[x,'Actors'] = response.get("Actors",'')
        tv_df.loc[x,'Plot'] = response.get("Plot",'')
        tv_df.loc[x,'Country'] = response.get("Country",'')
        tv_df.loc[x,'Awards'] = response.get("Awards",'')
        tv_df.loc[x,'Metascore'] = response.get("Metascore",'')
        tv_df.loc[x,'imdbRating'] = response.get("imdbRating",'')
        tv_df.loc[x,'imdbVotes'] = response.get("imdbVotes",'')
        tv_df.loc[x,'imdbID'] = response.get("imdbID",'')
        tv_df.loc[x,'Type'] = response.get("Type",'')
        
    except (KeyError, ValueError,JSONDecodeError) as e:
        print(f"skipping {tvid}")
        not_found.append(tvid)
        
    counter+=1        
    if counter == 10:
        break             
end_time = time.time()
elapsed = end_time - start_time
print(f"{elapsed} seconds elapsed, {len(not_found)} record(s) not found")
#tv_df.to_csv('.\\raw_data\\tv2018_raw.csv', encoding="utf-8", index=False)

1633.0402238368988 seconds elapsed, 0 record(s) not found


In [25]:
tv_df = pd.read_csv('.\\raw_data\\tv2018_raw.csv')

#Retain only needed column for tv analysis
tv_ratings = tv_df[['Title','Year','Genre','Country' ,'imdbRating','imdbVotes', 'imdbID']]

#Clean Year
tv_ratings = tv_ratings[tv_ratings["imdbRating"].notnull()]
tv_ratings["Year2"] = tv_ratings["Year"].apply(ratings_parse)

#Normalize imdb Ratings
tv_ratings = tv_ratings[tv_ratings["imdbRating"].notnull()]
tv_ratings["imdbRating2"] = tv_ratings["imdbRating"].multiply(10)

#Save as tv CSV
tv_ratings.to_csv('.\\cleaned_data\\tv_series_cleaned.csv', encoding="utf-8", index=False)
tv_ratings

,Title,Year,Genre,Country,imdbRating,imdbVotes,imdbID,Year2,imdbRating2
0,Zeg 'ns Aaa,1981–1993,Comedy,Netherlands,6.6,153,tt0062614,1981,66.0
1,Catweazle,1970–1971,"Adventure, Comedy, Family",UK,7.9,623,tt0063881,1970,79.0
2,UFO,1970–1973,"Action, Sci-Fi",UK,8.0,"2,333",tt0063962,1970,80.0
3,All My Children,1970–2011,"Drama, Romance, Mystery",USA,6.7,"2,128",tt0065272,1970,67.0
4,Archie's Funhouse,1970–1971,"Animation, Comedy, Family",USA,7.6,119,tt0065273,1970,76.0
5,Budgie,1971–1972,Drama,UK,7.9,133,tt0065279,1971,79.0
6,The Bugaloos,1970–1972,"Family, Fantasy",USA,7.0,330,tt0065280,1970,70.0
7,Dan August,1970–1971,"Crime, Drama",USA,7.1,175,tt0065286,1970,71.0
8,Doomwatch,1970–2016,"Sci-Fi, Drama",UK,7.3,110,tt0065290,1970,73.0
9,Flip,1970–1974,"Comedy, Music",USA,7.3,801,tt0065294,1970,73.0


In [26]:
#Open both movie files so 2018 can be appended to first list movies from first call
movie2018_file = pd.read_csv('.\\raw_data\\movies2018_raw.csv')
movie2018_data = pd.DataFrame(movie2018_file)

movie_file = pd.read_csv('.\\cleaned_data\\omdb_movies_cleaned.csv')
movie_data = pd.DataFrame(movie_file)

In [27]:
#Filter out titles that contain #DUPE#
movies2018 = movie2018_data[movie2018_data.Title != '#DUPE#']
#Retain only needed column for movie analysis
movie2018_data_df = movies2018[['Title','Year','Genre','Country','RT_Ratings','Metascore','imdbRating','imdbVotes','Type','Production']]

movie2018_df = movie2018_data_df[movie2018_data_df["RT_Ratings"].notnull()]

#Normalize Rotten Tomatoes ratings
movie2018_df["RT_Ratings2"] = movie2018_df["RT_Ratings"].apply(ratings_parse)

#Normalize imdb Ratings
movie2018_df["imdbRating2"] = movie2018_df["imdbRating"].multiply(10)

#Append to movie dataframe
allmovies = movie_data.append(movie2018_df)

#Save as movie combined CSV
allmovies.to_csv('.\\cleaned_data\\movies_w_2018_cleaned.csv', encoding="utf-8", index=False)

C:\Users\bethf\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\bethf\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


# Genre Cleanup (Movies)

In [28]:
movie_file = pd.read_csv('.\\cleaned_data\\movies_w_2018_cleaned.csv')
movie_data = pd.DataFrame(movie_file)

movie_data.head()

,Title,Year,Genre,Country,RT_Ratings,Metascore,imdbRating,imdbVotes,Type,Production,RT_Ratings2,imdbRating2
0,Toy Story,1995,"Animation, Adventure, Comedy, Family, Fantasy",USA,100%,95.0,8.3,"778,821",movie,Buena Vista,100,83.0
1,Jumanji,1995,"Adventure, Family, Fantasy",USA,53%,39.0,6.9,"264,569",movie,Sony Pictures Home Entertainment,53,69.0
2,Grumpier Old Men,1995,"Comedy, Romance",USA,17%,46.0,6.6,"22,256",movie,Warner Home Video,17,66.0
3,Waiting to Exhale,1995,"Comedy, Drama, Romance",USA,56%,NaN,5.8,"8,613",movie,Twentieth Century Fox Home Entertainment,56,58.0
4,Father of the Bride Part II,1995,"Comedy, Family, Romance",USA,48%,49.0,6.0,"30,949",movie,Disney,48,60.0


In [29]:
#Deal with multiple genres stores in single series
genres = movie_data.Genre.str.split(', ',n=10,expand=True)
genres.head()

,0,1,2,3,4,5,6,7,8,9
0,Animation,Adventure,Comedy,Family,Fantasy,None,None,None,None,None
1,Adventure,Family,Fantasy,None,None,None,None,None,None,None
2,Comedy,Romance,None,None,None,None,None,None,None,None
3,Comedy,Drama,Romance,None,None,None,None,None,None,None
4,Comedy,Family,Romance,None,None,None,None,None,None,None


In [30]:
#Make list of unique genres
genre_list = []
for row in range(17605):
    for column in range(10):
        if genres.iloc[row,column] not in genre_list:
            genre_list.append(genres.iloc[row,column])
#Delete 'nan' and 'none'
genre_list.pop(5)
genre_list.pop(-2) 
print(genre_list)

['Animation', 'Adventure', 'Comedy', 'Family', 'Fantasy', 'Romance', 'Drama', 'Crime', 'Thriller', 'Western', 'Action', 'Horror', 'History', 'Biography', 'Mystery', 'Sci-Fi', 'War', 'Musical', 'Sport', 'Music', 'Documentary', 'Short', 'News']


In [31]:
#Set genres that want to keep for analysis
#This is based on several experiments and looking at results of different combinations
#It is based on best judment call but has potential to skew analysis
genres_to_keep = ['Comedy','Fantasy','Drama','Action','Horror','Documentary','Animation']
len(genre_list)

23

In [32]:
movie_data_genre = movie_data[movie_data["Genre"].notnull()].reset_index(drop=True)

movie_data_genre.head()

,Title,Year,Genre,Country,RT_Ratings,Metascore,imdbRating,imdbVotes,Type,Production,RT_Ratings2,imdbRating2
0,Toy Story,1995,"Animation, Adventure, Comedy, Family, Fantasy",USA,100%,95.0,8.3,"778,821",movie,Buena Vista,100,83.0
1,Jumanji,1995,"Adventure, Family, Fantasy",USA,53%,39.0,6.9,"264,569",movie,Sony Pictures Home Entertainment,53,69.0
2,Grumpier Old Men,1995,"Comedy, Romance",USA,17%,46.0,6.6,"22,256",movie,Warner Home Video,17,66.0
3,Waiting to Exhale,1995,"Comedy, Drama, Romance",USA,56%,NaN,5.8,"8,613",movie,Twentieth Century Fox Home Entertainment,56,58.0
4,Father of the Bride Part II,1995,"Comedy, Family, Romance",USA,48%,49.0,6.0,"30,949",movie,Disney,48,60.0


In [33]:
#Find intersection of listed genres and genres to keep
def intersect(a, b):
    """ return the intersection of two lists as a comma separated string"""
    s = list(set(a) & set(b))
    new = ",".join(s)
    return new

for row in movie_data_genre.index:
    intersection = intersect(movie_data_genre.loc[row,"Genre"].split(', '),genres_to_keep)
    if len(intersection) >= 1:
        movie_data_genre.at[row,"Genre"] = intersection
    else:
        movie_data_genre.at[row,"Genre"] = "No Genre"

In [34]:
#View results
movie_genre = movie_data_genre[~movie_data_genre.Genre.str.contains("No Genre")].reset_index(drop=True)
print(movie_genre["Genre"].value_counts())

Drama                                     5597
Comedy,Drama                              2317
Comedy                                    1958
Documentary                               1674
Horror                                     871
Drama,Action                               814
Action                                     718
Comedy,Action                              385
Drama,Horror                               353
Comedy,Fantasy                             198
Comedy,Horror                              195
Fantasy,Drama                              178
Drama,Documentary                          136
Fantasy,Action                             132
Comedy,Animation,Fantasy                   128
Action,Horror                              125
Comedy,Fantasy,Drama                       124
Comedy,Animation                           111
Fantasy                                     96
Comedy,Documentary                          92
Comedy,Drama,Action                         88
Fantasy,Horro

In [37]:
#Written by Daniel Himmelstein https://stackoverflow.com/users/4651668/daniel-himmelstein
def tidy_split(df, column, sep='|', keep=False):
    """
    Split the values of a column and expand so the new DataFrame has one split
    value per row. Filters rows where the column is missing.

    Params
    ------
    df : pandas.DataFrame
        dataframe with the column to split and expand
    column : str
        the column to split and expand
    sep : str
        the string used to split the column's values
    keep : bool
        whether to retain the presplit value as it's own row

    Returns
    -------
    pandas.DataFrame
        Returns a dataframe with the same columns as `df`.
    """
    indexes = list()
    new_values = list()
    df = df.dropna(subset=[column])
    for i, presplit in enumerate(df[column].astype(str)):
        values = presplit.split(sep)
        if keep and len(values) > 1:
            indexes.append(i)
            new_values.append(presplit)
        for value in values:
            indexes.append(i)
            new_values.append(value)
    new_df = df.iloc[indexes, :].copy()
    new_df[column] = new_values
    return new_df

In [38]:
#"Explode" the movies that still have multiple genres to multiple rows for analysis
movies = tidy_split(movie_genre,'Genre',sep=',')
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24389 entries, 0 to 17157
Data columns (total 12 columns):
Title          24389 non-null object
Year           24389 non-null int64
Genre          24389 non-null object
Country        24384 non-null object
RT_Ratings     24389 non-null object
Metascore      16418 non-null float64
imdbRating     24389 non-null float64
imdbVotes      24388 non-null object
Type           24389 non-null object
Production     22747 non-null object
RT_Ratings2    24389 non-null int64
imdbRating2    24389 non-null float64
dtypes: float64(3), int64(2), object(7)
memory usage: 2.4+ MB


In [ ]:
#movies.to_csv('.\\cleaned_data\\movie_genres_cleaned.csv', encoding="utf-8", index=False)